In [5]:
import pandas as pd 
import json 

In [6]:
questions_df = pd.read_csv('../data/questions_df.csv')
questions_df = questions_df.query('corpus_id == "state_of_the_union"')[['question', 'references']]
questions_df.head()

,question,references
0,What significant regulatory changes and propos...,"[{""content"": ""My administration announced we\u..."
1,What reasons did President Biden give for the ...,"[{""content"": ""But unfortunately, politics have..."
2,How many people are no longer denied health in...,"[{""content"": ""Over 100 million of you can no l..."
3,"Which country is Putin invading, causing chaos...","[{""content"": ""Overseas, Putin of Russia is on ..."
4,When did the murder rate experience the sharpe...,"[{""content"": ""Last year, the murder rate saw t..."


In [7]:
for question, reference in questions_df.itertuples(index=False):
    print(f'Question: {question}')
    reference = json.loads(reference)
    print(f'Reference: {json.dumps(reference, indent=2)}')
    break

Question: What significant regulatory changes and proposals has President Biden's administration implemented or announced regarding fees and pricing transparency?
Reference: [
  {
    "content": "My administration announced we\u2019re cutting credit card late fees from $32 to $8.",
    "start_index": 27346,
    "end_index": 27425
  },
  {
    "content": "My administration has proposed rules to make cable, travel, utilities, and online ticket sellers tell you the total price up front so there are no surprises.",
    "start_index": 27866,
    "end_index": 28023
  }
]


In [8]:
from fixed_token_chunker import FixedTokenChunker

In [ ]:
chunker = FixedTokenChunker(reference)